### In the below code, we see the direct strategy applied using the hybrid CNN-LSTM model with attention. NASDAQ with symbol ^IXIC data is downloaded. In the first model, the data is first preporcessed using the direct strategy, sliding window method. The first difference is taken to make the data staionary, this helps the models forecast accuracy. The data is then split into test and train sets and fed into the neural network. The actual and predicted values are compared and the The root mean square error is 47 as seen below.

### Direct Strategy

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, BatchNormalization, Activation, Flatten, Reshape, Attention
import yfinance as yf
import numpy as np

# Get  stock data
data = yf.download('^IXIC', start='2005-01-01', end='2017-12-31').diff().dropna()

# Preprocess the data for LSTM
def preprocess_data(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps])
    X, y = np.array(X), np.array(y)
    return X, y



n_steps = 10  # number of time steps to consider for each sample
train_size = int(len(data)*0.8)
test_size = len(data) - train_size

X_train, y_train = preprocess_data(data['Close'].values[:train_size], n_steps)
X_test, y_test = preprocess_data(data['Close'].values[train_size:], n_steps)

# Define the Conv-LSTM-Att model with the direct strategy
inputs = tf.keras.layers.Input(shape=(n_steps, 1))
conv1d = Conv1D(32, kernel_size=3, padding='same', strides=1)(inputs)
lstm = LSTM(100, return_sequences=True)(conv1d)
query, value = tf.split(lstm, 2, axis=2)
attention = Attention(50)([query, value])
batch_norm1 = BatchNormalization()(attention)
dropout1 = Dropout(0.5)(batch_norm1)
dense1 = Dense(64)(dropout1)
batch_norm2 = BatchNormalization()(dense1)
dropout2 = Dropout(0.2)(batch_norm2)
output = Dense(1, activation='linear')(dropout2)
model_direct = Model(inputs=inputs, outputs=output)
model_direct.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')

# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_direct.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stopping])


[*********************100%%**********************]  1 of 1 completed
Epoch 1/10
163/163 [==============================] - 10s 17ms/step - loss: 1051.8833 - val_loss: 2228.1619
Epoch 2/10
163/163 [==============================] - 2s 14ms/step - loss: 1051.1376 - val_loss: 2227.1780
Epoch 3/10
163/163 [==============================] - 2s 15ms/step - loss: 1050.8947 - val_loss: 2226.8677
Epoch 4/10
163/163 [==============================] - 2s 14ms/step - loss: 1050.7260 - val_loss: 2226.3572
Epoch 5/10
163/163 [==============================] - 3s 19ms/step - loss: 1050.5156 - val_loss: 2225.9155
Epoch 6/10
163/163 [==============================] - 3s 17ms/step - loss: 1050.5148 - val_loss: 2225.6233
Epoch 7/10
163/163 [==============================] - 2s 14ms/step - loss: 1050.4896 - val_loss: 2225.6091
Epoch 8/10
163/163 [==============================] - 2s 14ms/step - loss: 1050.3994 - val_loss: 2225.3047
Epoch 9/10
163/163 [==============================] - 2s 15ms/step - loss:

In [ ]:
# Evaluate the model on the test data
rmse = model_direct.evaluate(X_test, y_test)
print("Root Mean Squared Error (RMSE):", np.sqrt(rmse))


21/21 [==============================] - 0s 17ms/step - loss: 2225.0569
Root Mean Squared Error (RMSE): 47.170508633738784


### In the below code, the data is not differenced and remains non stationary when it is fed into the model. This causes a drastic decrease in accuracy, increasing the root mean square error by over 5000.

## Direct Strategy Not diffferenced

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, BatchNormalization, Activation, Flatten, Reshape, Attention
import yfinance as yf
import numpy as np


# Get  stock data
data = yf.download('^IXIC', start='2005-01-01', end='2017-12-31')

# Preprocess the data for LSTM
def preprocess_data(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps])
    X, y = np.array(X), np.array(y)
    return X, y



n_steps = 10  # number of time steps to consider for each sample
train_size = int(len(data)*0.8)
test_size = len(data) - train_size

X_train, y_train = preprocess_data(data['Close'].values[:train_size], n_steps)
X_test, y_test = preprocess_data(data['Close'].values[train_size:], n_steps)

# Define the Conv-LSTM-Att model with the direct strategy
inputs = tf.keras.layers.Input(shape=(n_steps, 1))
conv1d = Conv1D(32, kernel_size=3, padding='same', strides=1)(inputs)
lstm = LSTM(100, return_sequences=True)(conv1d)

query, value = tf.split(lstm, 2, axis=2)
attention = Attention(50)([query, value])

batch_norm1 = BatchNormalization()(attention)
dropout1 = Dropout(0.5)(batch_norm1)
dense1 = Dense(64)(dropout1)
batch_norm2 = BatchNormalization()(dense1)
dropout2 = Dropout(0.2)(batch_norm2)
output = Dense(1, activation='linear')(dropout2)

model_direct = Model(inputs=inputs, outputs=output)

# Compile the model
model_direct.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')

# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_direct.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stopping])
# Evaluate the model on the test data
rmse = model_direct.evaluate(X_test, y_test)
print("Root Mean Squared Error (RMSE):", np.sqrt(rmse))

[*********************100%%**********************]  1 of 1 completed
Epoch 1/10
163/163 [==============================] - 8s 20ms/step - loss: 8408750.0000 - val_loss: 30355392.0000
Epoch 2/10
163/163 [==============================] - 2s 14ms/step - loss: 8363910.0000 - val_loss: 30214440.0000
Epoch 3/10
163/163 [==============================] - 2s 13ms/step - loss: 8270416.5000 - val_loss: 29976274.0000
Epoch 4/10
163/163 [==============================] - 2s 14ms/step - loss: 8130110.5000 - val_loss: 29629464.0000
Epoch 5/10
163/163 [==============================] - 3s 17ms/step - loss: 7948677.5000 - val_loss: 29230304.0000
Epoch 6/10
163/163 [==============================] - 3s 19ms/step - loss: 7730552.5000 - val_loss: 28773790.0000
Epoch 7/10
163/163 [==============================] - 2s 14ms/step - loss: 7480912.0000 - val_loss: 28241686.0000
Epoch 8/10
163/163 [==============================] - 2s 14ms/step - loss: 7204506.5000 - val_loss: 27647180.0000
Epoch 9/10
163/163 